In [76]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

df = pd.read_parquet('./data')
print(f'total dataset: {len(df)} rows')

## filters
# df = df[df['stabilized_spread'] < 10]  # nem tul nagy spread
# df = df[df['stabilized_spread'] > 0.2]  # nem tul nagy spread
# df = df[abs(df['last_price_delta_since_stabilized']) > 0.1]
# df = df[abs(df['last_price_delta_since_stabilized']) < 8]

# df = df[df['stabilized_amount_mean'] < 0.4]  # nem tul nagy tradek
# df = df[df['stabilized_nr_trades'] < 300]  # nem tul sok trade

df.sort_index(axis=1, inplace=True)

y = df['last_price_delta_since_stabilized']
X = df.drop('last_price_delta_since_stabilized', axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)
X_train

total dataset: 23000 rows


,0_amount_mean,0_nr_trades,0_price_delta,0_spread,1_amount_mean,1_nr_trades,1_price_delta,1_spread,2_amount_mean,2_nr_trades,...,past_final_price_95,past_final_price_96,past_final_price_97,past_final_price_98,past_final_price_99,stabilized_amount_mean,stabilized_at_ms,stabilized_nr_trades,stabilized_spread,wave_direction
index,,,,,,,,,,,,,,,,,,,,,
0,0.0337,8.0,0.02,1.13,0.0300,9.0,0.02,1.13,0.0359,10.0,...,-2.18,-1.02,-0.84,-0.02,0.0,0.0344,544,13.0,1.15,max
0,0.0629,4.0,0.47,0.45,0.0440,6.0,0.46,0.47,0.0380,7.0,...,2.44,2.41,0.40,1.40,0.0,0.0269,97,10.0,0.93,min
0,0.0033,12.0,0.07,0.79,0.0036,13.0,0.07,0.79,0.0112,14.0,...,2.62,0.29,0.91,-1.89,0.0,0.0123,712,17.0,0.86,max
0,0.0034,3.0,0.32,0.04,0.0054,4.0,0.32,0.08,0.0044,5.0,...,-0.37,-0.17,0.26,1.11,0.0,0.0037,341,8.0,0.40,None
0,0.0067,17.0,0.37,0.38,0.0064,18.0,0.37,0.38,0.0062,19.0,...,1.51,0.71,0.67,0.42,0.0,0.0078,85,22.0,0.75,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0019,3.0,0.18,0.11,0.0014,4.0,0.17,0.12,0.0013,5.0,...,2.15,-0.02,1.19,0.51,0.0,0.0126,41,8.0,0.29,None
0,0.0446,214.0,0.20,5.72,0.0448,215.0,0.20,5.72,0.0455,216.0,...,1.95,-0.54,-1.00,0.76,0.0,0.0439,718,224.0,5.92,min
0,0.0118,8.0,0.40,0.30,0.0111,9.0,0.40,0.30,0.0605,33.0,...,2.78,2.70,3.13,4.78,0.0,0.0749,53,39.0,0.70,min


In [77]:
from spreadsurfer.price_engine import FeatureEngineer
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
# from sklearn.ensemle import RandomForestRegressor

model = CatBoostRegressor(learning_rate=0.015, depth=7, loss_function='RMSE', random_state=0, verbose=False, iterations=1500)

pipeline = Pipeline(steps=[
    ('preprocessor', FeatureEngineer()),
    # ('scaler', StandardScaler()),
    # ('model', RandomForestRegressor(n_estimators=50, random_state=0))
    ('model', model)
])

pipeline.fit(X_train, y_train)
score = pipeline.score(X_valid, y_valid)
display(score)


0.44225830477346706

In [517]:
import catboost
w = catboost.MetricVisualizer('./models', subdirs=True)
w.start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

boo


In [427]:
from sklearn.model_selection import GridSearchCV

do_grid_search = False

if do_grid_search:
    X = FeatureEngineer().transform(X)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

    parameters = {'depth': [7, 8],
                  'learning_rate': [0.015, 0.03, 0.04],
                  'iterations': [100, 1000, 1500]
                  }
    grid = GridSearchCV(estimator=model, param_grid=parameters, cv=2, n_jobs=-1)
    grid.fit(X_train, y_train)

    print(" Results from Grid Search ")
    print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
    print("\n The best score across ALL searched params:\n", grid.best_score_)
    print("\n The best parameters across ALL searched params:\n", grid.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:

 The best score across ALL searched params:
 0.5933380986132425

 The best parameters across ALL searched params:
 {'depth': 7, 'iterations': 1500, 'learning_rate': 0.015}


In [26]:
from spreadsurfer import now_isoformat

save = True
if save:
    model.save_model(f'./models/{now_isoformat()}-score-{1000 * round(score, 3)}.cat')


In [78]:
input_from_log = """

2023-01-24 15:20:37.288 | data     | spreadsurfer.datacollect:start:93 - wave 23444 collected: {'0_amount_mean': 0.0474, '0_nr_trades': 69.0, '0_price_delta': 1.01, '0_spread': 2.36, '1_amount_mean': 0.0483, '1_nr_trades': 80.0, '1_price_delta': 0.47, '1_spread': 2.9, '2_amount_mean': 0.0519, '2_nr_trades': 92.0, '2_price_delta': 0.0, '2_spread': 3.37, '3_amount_mean': 0.048, '3_nr_trades': 110.0, '3_price_delta': 0.0, '3_spread': 3.37, '4_amount_mean': 0.0475, '4_nr_trades': 111.0, '4_price_delta': 0.0, '4_spread': 3.37, 'last_price_delta_since_stabilized': 0.0, 'past_final_price_0': 5.400000000001455, 'past_final_price_1': 2.650000000001455, 'past_final_price_10': 21.80000000000291, 'past_final_price_11': 23.55000000000291, 'past_final_price_12': 18.960000000002765, 'past_final_price_13': 17.7300000000032, 'past_final_price_14': 15.080000000001746, 'past_final_price_15': 9.670000000001892, 'past_final_price_16': 9.460000000002765, 'past_final_price_17': 10.909999999999854, 'past_final_price_18': 8.400000000001455, 'past_final_price_19': 7.30000000000291, 'past_final_price_2': 4.280000000002474, 'past_final_price_20': 5.270000000000437, 'past_final_price_21': 7.890000000003056, 'past_final_price_22': 8.590000000000146, 'past_final_price_23': 7.190000000002328, 'past_final_price_24': 5.819999999999709, 'past_final_price_25': 6.400000000001455, 'past_final_price_26': 7.970000000001164, 'past_final_price_27': -0.47999999999956344, 'past_final_price_28': 0.680000000000291, 'past_final_price_29': -1.75, 'past_final_price_3': 4.400000000001455, 'past_final_price_30': -1.4199999999982538, 'past_final_price_31': -1.8099999999976717, 'past_final_price_32': 0.1900000000023283, 'past_final_price_33': 0.21000000000276486, 'past_final_price_34': 3.9900000000016007, 'past_final_price_35': 8.190000000002328, 'past_final_price_36': 11.37000000000262, 'past_final_price_37': 9.780000000002474, 'past_final_price_38': nan, 'past_final_price_39': -11.37999999999738, 'past_final_price_4': 7.620000000002619, 'past_final_price_40': -8.209999999999127, 'past_final_price_41': -7.1599999999998545, 'past_final_price_42': -5.430000000000291, 'past_final_price_43': -2.7999999999992724, 'past_final_price_44': -1.7899999999972351, 'past_final_price_45': -1.319999999999709, 'past_final_price_46': -1.5999999999985448, 'past_final_price_47': nan, 'past_final_price_48': -1.319999999999709, 'past_final_price_49': -2.899999999997817, 'past_final_price_5': 10.340000000000146, 'past_final_price_50': 1.1300000000010186, 'past_final_price_51': -3.5, 'past_final_price_52': 2.0800000000017462, 'past_final_price_53': -0.6899999999986903, 'past_final_price_54': 2.9500000000007276, 'past_final_price_55': 3.400000000001455, 'past_final_price_56': 9.640000000003056, 'past_final_price_57': 8.730000000003201, 'past_final_price_58': 9.610000000000582, 'past_final_price_59': 7.100000000002183, 'past_final_price_6': 14.100000000002183, 'past_final_price_60': 13.100000000002183, 'past_final_price_61': 8.100000000002183, 'past_final_price_62': 8.340000000000146, 'past_final_price_63': 6.580000000001746, 'past_final_price_64': 4.110000000000582, 'past_final_price_65': 4.260000000002037, 'past_final_price_66': -1.0399999999972351, 'past_final_price_67': -2.6699999999982538, 'past_final_price_68': -5.729999999999563, 'past_final_price_69': -10.059999999997672, 'past_final_price_7': 16.63000000000102, 'past_final_price_70': -8.979999999999563, 'past_final_price_71': -8.269999999996799, 'past_final_price_72': -6.789999999997235, 'past_final_price_73': -9.649999999997817, 'past_final_price_74': -9.5099999999984, 'past_final_price_75': -5.809999999997672, 'past_final_price_76': -10.12999999999738, 'past_final_price_77': -7.6599999999998545, 'past_final_price_78': -8.75, 'past_final_price_79': -9.369999999998981, 'past_final_price_8': 18.390000000003056, 'past_final_price_80': -7.3400000000001455, 'past_final_price_81': -11.0, 'past_final_price_82': -8.709999999999127, 'past_final_price_83': -8.719999999997526, 'past_final_price_84': -8.359999999996944, 'past_final_price_85': -8.359999999996944, 'past_final_price_86': -7.349999999998545, 'past_final_price_87': -6.639999999999418, 'past_final_price_88': -3.889999999999418, 'past_final_price_89': -5.019999999996799, 'past_final_price_9': 21.400000000001455, 'past_final_price_90': -4.379999999997381, 'past_final_price_91': -7.1599999999998545, 'past_final_price_92': -6.639999999999418, 'past_final_price_93': -6.849999999998545, 'past_final_price_94': -4.0900000000001455, 'past_final_price_95': -2.7299999999995634, 'past_final_price_96': -2.149999999997817, 'past_final_price_97': -6.180000000000291, 'past_final_price_98': 0.9099999999998545, 'past_final_price_99': 0.0, 'stabilized_amount_mean': 0.0467, 'stabilized_at_ms': 683, 'stabilized_nr_trades': 113.0, 'stabilized_spread': 3.37, 'wave_direction': 'max'}
2023-01-24 15:20:39.393 | data     | spreadsurfer.datacollect:start:93 - wave 23445 collected: {'0_amount_mean': 0.0085, '0_nr_trades': 4.0, '0_price_delta': 1.72, '0_spread': 0.03, '1_amount_mean': 0.0093, '1_nr_trades': 5.0, '1_price_delta': 1.72, '1_spread': 0.03, '2_amount_mean': 0.0125, '2_nr_trades': 7.0, '2_price_delta': 1.7, '2_spread': 0.05, '3_amount_mean': 0.0216, '3_nr_trades': 16.0, '3_price_delta': 0.94, '3_spread': 0.81, '4_amount_mean': 0.0183, '4_nr_trades': 30.0, '4_price_delta': 0.88, '4_spread': 0.87, 'last_price_delta_since_stabilized': 0.04, 'past_final_price_0': 9.880000000001019, 'past_final_price_1': 10.0, 'past_final_price_10': 24.56000000000131, 'past_final_price_11': 23.330000000001746, 'past_final_price_12': 20.68000000000029, 'past_final_price_13': 15.270000000000437, 'past_final_price_14': 15.06000000000131, 'past_final_price_15': 16.5099999999984, 'past_final_price_16': 14.0, 'past_final_price_17': 12.900000000001455, 'past_final_price_18': 10.869999999998981, 'past_final_price_19': 13.4900000000016, 'past_final_price_2': 13.220000000001164, 'past_final_price_20': 14.18999999999869, 'past_final_price_21': 12.790000000000873, 'past_final_price_22': 11.419999999998254, 'past_final_price_23': 12.0, 'past_final_price_24': 13.569999999999709, 'past_final_price_25': 5.119999999998981, 'past_final_price_26': 6.279999999998836, 'past_final_price_27': 3.849999999998545, 'past_final_price_28': 4.180000000000291, 'past_final_price_29': 3.790000000000873, 'past_final_price_3': 15.93999999999869, 'past_final_price_30': 5.790000000000873, 'past_final_price_31': 5.81000000000131, 'past_final_price_32': 9.590000000000146, 'past_final_price_33': 13.790000000000873, 'past_final_price_34': 16.970000000001164, 'past_final_price_35': 15.380000000001019, 'past_final_price_36': nan, 'past_final_price_37': -5.779999999998836, 'past_final_price_38': -2.610000000000582, 'past_final_price_39': -1.5600000000013097, 'past_final_price_4': 19.700000000000728, 'past_final_price_40': 0.16999999999825377, 'past_final_price_41': 2.7999999999992724, 'past_final_price_42': 3.8100000000013097, 'past_final_price_43': 4.279999999998836, 'past_final_price_44': 4.0, 'past_final_price_45': nan, 'past_final_price_46': 4.279999999998836, 'past_final_price_47': 2.7000000000007276, 'past_final_price_48': 6.729999999999563, 'past_final_price_49': 2.099999999998545, 'past_final_price_5': 22.229999999999563, 'past_final_price_50': 7.680000000000291, 'past_final_price_51': 4.9099999999998545, 'past_final_price_52': 8.549999999999272, 'past_final_price_53': 9.0, 'past_final_price_54': 15.2400000000016, 'past_final_price_55': 14.330000000001746, 'past_final_price_56': 15.209999999999127, 'past_final_price_57': 12.700000000000728, 'past_final_price_58': 18.700000000000728, 'past_final_price_59': 13.700000000000728, 'past_final_price_6': 23.9900000000016, 'past_final_price_60': 13.93999999999869, 'past_final_price_61': 12.180000000000291, 'past_final_price_62': 9.709999999999127, 'past_final_price_63': 9.860000000000582, 'past_final_price_64': 4.56000000000131, 'past_final_price_65': 2.930000000000291, 'past_final_price_66': -0.13000000000101863, 'past_final_price_67': -4.459999999999127, 'past_final_price_68': -3.3800000000010186, 'past_final_price_69': -2.6699999999982538, 'past_final_price_7': 27.0, 'past_final_price_70': -1.1899999999986903, 'past_final_price_71': -4.049999999999272, 'past_final_price_72': -3.9099999999998545, 'past_final_price_73': -0.20999999999912689, 'past_final_price_74': -4.529999999998836, 'past_final_price_75': -2.0600000000013097, 'past_final_price_76': -3.150000000001455, 'past_final_price_77': -3.7700000000004366, 'past_final_price_78': -1.7400000000016007, 'past_final_price_79': -5.400000000001455, 'past_final_price_8': 27.400000000001455, 'past_final_price_80': -3.110000000000582, 'past_final_price_81': -3.1199999999989814, 'past_final_price_82': -2.7599999999983993, 'past_final_price_83': -2.7599999999983993, 'past_final_price_84': -1.75, 'past_final_price_85': -1.0400000000008731, 'past_final_price_86': 1.7099999999991269, 'past_final_price_87': 0.5800000000017462, 'past_final_price_88': 1.2200000000011642, 'past_final_price_89': -1.5600000000013097, 'past_final_price_9': 29.150000000001455, 'past_final_price_90': -1.0400000000008731, 'past_final_price_91': -1.25, 'past_final_price_92': 1.5099999999983993, 'past_final_price_93': 2.8699999999989814, 'past_final_price_94': 3.4500000000007276, 'past_final_price_95': -0.5800000000017462, 'past_final_price_96': 6.509999999998399, 'past_final_price_97': 5.599999999998545, 'past_final_price_98': 3.9500000000007276, 'past_final_price_99': 0.0, 'stabilized_amount_mean': 0.0115, 'stabilized_at_ms': 789, 'stabilized_nr_trades': 56.0, 'stabilized_spread': 1.75, 'wave_direction': 'min'}
2023-01-24 15:20:41.153 | data     | spreadsurfer.datacollect:start:93 - wave 23446 collected: {'0_amount_mean': 0.0273, '0_nr_trades': 11.0, '0_price_delta': 2.82, '0_spread': 0.86, '1_amount_mean': 0.0229, '1_nr_trades': 33.0, '1_price_delta': 2.51, '1_spread': 1.17, '2_amount_mean': 0.0244, '2_nr_trades': 110.0, '2_price_delta': 0.0, '2_spread': 3.68, '3_amount_mean': 0.0277, '3_nr_trades': 124.0, '3_price_delta': 0.0, '3_spread': 3.68, '4_amount_mean': 0.0275, '4_nr_trades': 125.0, '4_price_delta': 0.0, '4_spread': 3.68, 'last_price_delta_since_stabilized': 0.0, 'past_final_price_0': 19.159999999999854, 'past_final_price_1': 21.87999999999738, 'past_final_price_10': 26.61999999999898, 'past_final_price_11': 21.209999999999127, 'past_final_price_12': 21.0, 'past_final_price_13': 22.44999999999709, 'past_final_price_14': 19.93999999999869, 'past_final_price_15': 18.840000000000146, 'past_final_price_16': 16.80999999999767, 'past_final_price_17': 19.43000000000029, 'past_final_price_18': 20.12999999999738, 'past_final_price_19': 18.729999999999563, 'past_final_price_2': 25.639999999999418, 'past_final_price_20': 17.359999999996944, 'past_final_price_21': 17.93999999999869, 'past_final_price_22': 19.5099999999984, 'past_final_price_23': 11.059999999997672, 'past_final_price_24': 12.219999999997526, 'past_final_price_25': 9.789999999997235, 'past_final_price_26': 10.119999999998981, 'past_final_price_27': 9.729999999999563, 'past_final_price_28': 11.729999999999563, 'past_final_price_29': 11.75, 'past_final_price_3': 28.169999999998254, 'past_final_price_30': 15.529999999998836, 'past_final_price_31': 19.729999999999563, 'past_final_price_32': 22.909999999999854, 'past_final_price_33': 21.31999999999971, 'past_final_price_34': nan, 'past_final_price_35': 0.15999999999985448, 'past_final_price_36': 3.3299999999981083, 'past_final_price_37': 4.379999999997381, 'past_final_price_38': 6.109999999996944, 'past_final_price_39': 8.739999999997963, 'past_final_price_4': 29.93000000000029, 'past_final_price_40': 9.75, 'past_final_price_41': 10.219999999997526, 'past_final_price_42': 9.93999999999869, 'past_final_price_43': nan, 'past_final_price_44': 10.219999999997526, 'past_final_price_45': 8.639999999999418, 'past_final_price_46': 12.669999999998254, 'past_final_price_47': 8.039999999997235, 'past_final_price_48': 13.619999999998981, 'past_final_price_49': 10.849999999998545, 'past_final_price_5': 32.93999999999869, 'past_final_price_50': 14.489999999997963, 'past_final_price_51': 14.93999999999869, 'past_final_price_52': 21.18000000000029, 'past_final_price_53': 20.270000000000437, 'past_final_price_54': 21.149999999997817, 'past_final_price_55': 18.639999999999418, 'past_final_price_56': 24.639999999999418, 'past_final_price_57': 19.639999999999418, 'past_final_price_58': 19.87999999999738, 'past_final_price_59': 18.11999999999898, 'past_final_price_6': 33.340000000000146, 'past_final_price_60': 15.649999999997817, 'past_final_price_61': 15.799999999999272, 'past_final_price_62': 10.5, 'past_final_price_63': 8.869999999998981, 'past_final_price_64': 5.809999999997672, 'past_final_price_65': 1.4799999999995634, 'past_final_price_66': 2.5599999999976717, 'past_final_price_67': 3.2700000000004366, 'past_final_price_68': 4.75, 'past_final_price_69': 1.889999999999418, 'past_final_price_7': 35.090000000000146, 'past_final_price_70': 2.029999999998836, 'past_final_price_71': 5.729999999999563, 'past_final_price_72': 1.4099999999998545, 'past_final_price_73': 3.8799999999973807, 'past_final_price_74': 2.789999999997235, 'past_final_price_75': 2.1699999999982538, 'past_final_price_76': 4.19999999999709, 'past_final_price_77': 0.5399999999972351, 'past_final_price_78': 2.8299999999981083, 'past_final_price_79': 2.819999999999709, 'past_final_price_8': 30.5, 'past_final_price_80': 3.180000000000291, 'past_final_price_81': 3.180000000000291, 'past_final_price_82': 4.18999999999869, 'past_final_price_83': 4.899999999997817, 'past_final_price_84': 7.649999999997817, 'past_final_price_85': 6.520000000000437, 'past_final_price_86': 7.1599999999998545, 'past_final_price_87': 4.379999999997381, 'past_final_price_88': 4.899999999997817, 'past_final_price_89': 4.68999999999869, 'past_final_price_9': 29.270000000000437, 'past_final_price_90': 7.44999999999709, 'past_final_price_91': 8.809999999997672, 'past_final_price_92': 9.389999999999418, 'past_final_price_93': 5.359999999996944, 'past_final_price_94': 12.44999999999709, 'past_final_price_95': 11.539999999997235, 'past_final_price_96': 9.889999999999418, 'past_final_price_97': 5.93999999999869, 'past_final_price_98': 4.5, 'past_final_price_99': 0.0, 'stabilized_amount_mean': 0.0271, 'stabilized_at_ms': 682, 'stabilized_nr_trades': 133.0, 'stabilized_spread': 3.68, 'wave_direction': 'min'}
2023-01-24 15:20:43.193 | data     | spreadsurfer.datacollect:start:93 - wave 23447 collected: {'0_amount_mean': 0.0013, '0_nr_trades': 4.0, '0_price_delta': 0.94, '0_spread': 0.45, '1_amount_mean': 0.0047, '1_nr_trades': 5.0, '1_price_delta': 0.89, '1_spread': 0.5, '2_amount_mean': 0.0041, '2_nr_trades': 6.0, '2_price_delta': 0.89, '2_spread': 0.5, '3_amount_mean': 0.0048, '3_nr_trades': 8.0, '3_price_delta': 0.89, '3_spread': 0.5, '4_amount_mean': 0.0086, '4_nr_trades': 13.0, '4_price_delta': 0.0, '4_spread': 1.39, 'last_price_delta_since_stabilized': -0.29, 'past_final_price_0': 23.610000000000582, 'past_final_price_1': 26.139999999999418, 'past_final_price_10': 18.970000000001164, 'past_final_price_11': 20.419999999998254, 'past_final_price_12': 17.909999999999854, 'past_final_price_13': 16.81000000000131, 'past_final_price_14': 14.779999999998836, 'past_final_price_15': 17.400000000001455, 'past_final_price_16': 18.099999999998545, 'past_final_price_17': 16.700000000000728, 'past_final_price_18': 15.329999999998108, 'past_final_price_19': 15.909999999999854, 'past_final_price_2': 27.900000000001455, 'past_final_price_20': 17.479999999999563, 'past_final_price_21': 9.029999999998836, 'past_final_price_22': 10.18999999999869, 'past_final_price_23': 7.759999999998399, 'past_final_price_24': 8.090000000000146, 'past_final_price_25': 7.700000000000728, 'past_final_price_26': 9.700000000000728, 'past_final_price_27': 9.720000000001164, 'past_final_price_28': 13.5, 'past_final_price_29': 17.700000000000728, 'past_final_price_3': 30.909999999999854, 'past_final_price_30': 20.88000000000102, 'past_final_price_31': 19.290000000000873, 'past_final_price_32': nan, 'past_final_price_33': -1.8699999999989814, 'past_final_price_34': 1.2999999999992724, 'past_final_price_35': 2.349999999998545, 'past_final_price_36': 4.079999999998108, 'past_final_price_37': 6.709999999999127, 'past_final_price_38': 7.720000000001164, 'past_final_price_39': 8.18999999999869, 'past_final_price_4': 31.31000000000131, 'past_final_price_40': 7.9099999999998545, 'past_final_price_41': nan, 'past_final_price_42': 8.18999999999869, 'past_final_price_43': 6.610000000000582, 'past_final_price_44': 10.639999999999418, 'past_final_price_45': 6.009999999998399, 'past_final_price_46': 11.590000000000146, 'past_final_price_47': 8.819999999999709, 'past_final_price_48': 12.459999999999127, 'past_final_price_49': 12.909999999999854, 'past_final_price_5': 33.06000000000131, 'past_final_price_50': 19.150000000001455, 'past_final_price_51': 18.2400000000016, 'past_final_price_52': 19.11999999999898, 'past_final_price_53': 16.610000000000582, 'past_final_price_54': 22.610000000000582, 'past_final_price_55': 17.610000000000582, 'past_final_price_56': 17.849999999998545, 'past_final_price_57': 16.090000000000146, 'past_final_price_58': 13.619999999998981, 'past_final_price_59': 13.770000000000437, 'past_final_price_6': 28.470000000001164, 'past_final_price_60': 8.470000000001164, 'past_final_price_61': 6.8400000000001455, 'past_final_price_62': 3.779999999998836, 'past_final_price_63': -0.5499999999992724, 'past_final_price_64': 0.5299999999988358, 'past_final_price_65': 1.2400000000016007, 'past_final_price_66': 2.720000000001164, 'past_final_price_67': -0.13999999999941792, 'past_final_price_68': 0.0, 'past_final_price_69': 3.7000000000007276, 'past_final_price_7': 27.2400000000016, 'past_final_price_70': -0.6199999999989814, 'past_final_price_71': 1.8499999999985448, 'past_final_price_72': 0.7599999999983993, 'past_final_price_73': 0.13999999999941792, 'past_final_price_74': 2.1699999999982538, 'past_final_price_75': -1.4900000000016007, 'past_final_price_76': 0.7999999999992724, 'past_final_price_77': 0.7900000000008731, 'past_final_price_78': 1.1500000000014552, 'past_final_price_79': 1.1500000000014552, 'past_final_price_8': 24.590000000000146, 'past_final_price_80': 2.1599999999998545, 'past_final_price_81': 2.8699999999989814, 'past_final_price_82': 5.619999999998981, 'past_final_price_83': 4.490000000001601, 'past_final_price_84': 5.130000000001019, 'past_final_price_85': 2.349999999998545, 'past_final_price_86': 2.8699999999989814, 'past_final_price_87': 2.6599999999998545, 'past_final_price_88': 5.419999999998254, 'past_final_price_89': 6.779999999998836, 'past_final_price_9': 19.18000000000029, 'past_final_price_90': 7.360000000000582, 'past_final_price_91': 3.3299999999981083, 'past_final_price_92': 10.419999999998254, 'past_final_price_93': 9.5099999999984, 'past_final_price_94': 7.860000000000582, 'past_final_price_95': 3.9099999999998545, 'past_final_price_96': 2.470000000001164, 'past_final_price_97': -2.029999999998836, 'past_final_price_98': 0.7599999999983993, 'past_final_price_99': 0.0, 'stabilized_amount_mean': 0.0081, 'stabilized_at_ms': 517, 'stabilized_nr_trades': 14.0, 'stabilized_spread': 1.39, 'wave_direction': 'max'}




"""
from timeit import timeit

mae = 0
count = 0
for line in input_from_log.split('\n'):
    if not line: continue
    count += 1
    json = '{' + line.split('{')[1]
    sample = pd.DataFrame([eval(json)])

    real = sample.last_price_delta_since_stabilized[0]
    print('expected: ', real)
    sample.drop('last_price_delta_since_stabilized', axis=1, inplace=True)

    timeit(lambda: pipeline.predict(sample), number=1) * 1000
    guess = pipeline.predict(sample)[0]
    print('predict: ', guess)
    mae += abs(real - guess)
    print()
display(f'MAE: {mae / count}')

NameError: name 'nan' is not defined

In [438]:
sample = pd.DataFrame([
    {'0_amount_mean': 0.007, '0_nr_trades': 12.0, '0_price_delta': 0.91, '0_spread': 1.15, '1_amount_mean': 0.007, '1_nr_trades': 13.0, '1_price_delta': 0.91, '1_spread': 1.52, '2_amount_mean': 0.006, '2_nr_trades': 14.0,
     '2_price_delta': 0.91, '2_spread': 1.52, '3_amount_mean': 0.006, '3_nr_trades': 16.0, '3_price_delta': 0.69, '3_spread': 1.74, '4_amount_mean': 0.005, '4_nr_trades': 17.0, '4_price_delta': 0.69, '4_spread': 1.74,
     'last_price_delta_since_stabilized': -0.87, 'past_final_price_0': -0.11999999999898137, 'past_final_price_1': 0.25, 'past_final_price_2': -0.020000000000436557, 'past_final_price_3': -1.7400000000016007,
     'past_final_price_4': 0.06999999999970896, 'past_final_price_5': -2.7299999999995634, 'past_final_price_6': -3.9099999999998545, 'past_final_price_7': 3.360000000000582, 'past_final_price_8': -0.06000000000130967,
     'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.007, 'stabilized_at_ms': 369, 'stabilized_nr_trades': 18.0, 'stabilized_spread': 2.43, 'wave_direction': 'max'}
])

print('expected: ', sample.last_price_delta_since_stabilized.real())
sample.drop('last_price_delta_since_stabilized', axis=1, inplace=True)

from timeit import timeit

timeit(lambda: pipeline.predict(sample), number=1) * 1000
print('predict: ', pipeline.predict(sample)[0])


expected:  -0.87
predict:  -0.9450021282696802


In [278]:
from glob import glob

cat_filename = glob('*.cat')[0]
cat_filename

model = CatBoostRegressor()
model.load_model(fname=cat_filename)
print(f'catboost model loaded from {cat_filename}')


catboost model loaded from 2023-01-21T20:37:02-score-773.0.cat
